# Brief analysis of the US consumer price index (CPI) 

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import signal

In [ ]:
# Get US CPI and fed funds effective rate dfs
df_cpi = pd.read_csv("../data/US_CPI_M.csv")
df_fed = pd.read_csv("../data/FED_FUNDS_EFFECTIVE_RATE_M.csv")

In [ ]:
# Append the CPI estimate for April of 2024 to the CPI df
new_row = pd.DataFrame({"year_month": ["2024-04"], "rate": [3.4]})
df_cpi = pd.concat([df_cpi, new_row], ignore_index=True)

In [ ]:
# Set date as index for both dfs
for df in df_cpi, df_fed:
    df["year_month"] = pd.to_datetime(df["year_month"])
    df.set_index("year_month", inplace=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

In [ ]:
# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

## Datasets basic info

### US consumer price index (CPI)

In [ ]:
# First and last entries
pd.concat([df_cpi.head(1), df_cpi.tail(1)]).T

In [ ]:
df_cpi.describe().T

### Federal Reserve funds effective rate

In [ ]:
# First and last entries
pd.concat([df_fed.head(1), df_fed.tail(1)]).T

In [ ]:
df_fed.describe().T

## CPI long-term analysis

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_cpi, x=df_cpi.index, y="rate", label="CPI", color="lime", linewidth=0.75)
plt.axhline(y=2, label="Fed 2% target", color="red", linewidth=1)

plt.title("US consumer price index (CPI) over time")
plt.xlabel("Date")
plt.ylabel("Rate (%)")
plt.legend()

plt.show()

In [ ]:
# All time high
ath_ym = df_cpi["rate"].idxmax().strftime("%Y-%m")
df_cpi.loc[ath_ym]

In [ ]:
# All time low
atl_ym = df_cpi["rate"].idxmin().strftime("%Y-%m")
df_cpi.loc[atl_ym]

In [ ]:
# Highest 6 peaks
peaks, _ = signal.find_peaks(df_cpi["rate"], distance=36)
df_cpi.iloc[peaks].nlargest(6, "rate").sort_values("year_month").T

In [ ]:
# Lowest 6 valleys
valleys, _ = signal.find_peaks(-df_cpi["rate"], distance=36)
df_cpi.iloc[valleys].nsmallest(6, "rate").sort_values("year_month").T

In [ ]:
# Get deflationary periods table
# Get negative rates df
df_cpi_def = df_cpi.loc[df_cpi["rate"] < 0].copy()
# Get periods of consecutive dates
df_cpi_def["period"] = (df_cpi_def.index.diff().days > 31).cumsum() + 1
# Get begin, end and avg rate of each period into another df
df_cpi_def.groupby("period").agg(
    begin_date=("rate", lambda x: x.index.min()),
    end_date=("rate", lambda x: x.index.max()),
    average_rate=("rate", "mean")
)

In [ ]:
# Get longest 5 periods (and respective avg rate) in which the rate was closest to the 2% target (with a 1 percentage point deviation margin)
# Get df with rates close to 2%
df_cpi_close_to_2 = df_cpi[df_cpi["rate"].sub(2).abs().lt(1)].copy()
# Get periods of consecutive dates
df_cpi_close_to_2["period"] = (df_cpi_close_to_2.index.diff().days > 31).cumsum() + 1
# Get begin, end and avg rate of each period into another df
df_close_to_2_periods = df_cpi_close_to_2.groupby("period").agg(
    begin_date=("rate", lambda x: x.index.min()),
    end_date=("rate", lambda x: x.index.max()),
    average_rate=("rate", "mean")
)
# Get the 5 longest periods
longest_periods = df_cpi_close_to_2["period"].value_counts().nlargest(5).index
df_close_to_2_periods.loc[longest_periods]

In [ ]:
# Get average inflation rate of each decade
df_cpi_dec = df_cpi.groupby((df_cpi.index.year // 10) * 10)
df_cpi_dec = df_cpi_dec["rate"].mean().round(2).reset_index()
df_cpi_dec.columns = ["decade", "avg_cpi"]
df_cpi_dec.set_index("decade").T

- inflation started rising in the second half of 60s, up until 80 all time high. then quickly fell.
- only during the first hald of the 60s inflation was below the 2% target (considered a ceiling in the past).
- also during the 2010s until the end of the pandemic lockdowns inflation was sort of contained below that.
- most of the time, especially during the 70s and early 80s, inflation rate was really high.
- deflation only on 2009 and 2015.

## CPI + fed rates since 2020 analysis

In [ ]:
# Get dfs since 2020
df_cpi_2020 = df_cpi["2020":].copy()
df_fed_2020 = df_fed["2020":].copy()

In [ ]:
plt.figure(figsize=(14, 8))

sns.lineplot(data=df_cpi_2020, x=df_cpi_2020.index, y="rate", label="CPI", color="lime", linewidth=1)
sns.lineplot(data=df_fed_2020, x=df_fed_2020.index, y="rate", label="Fed funds effective rate", color="violet", linewidth=1)
plt.axhline(y=2, label="Fed 2% target", color="red", linewidth=2)

plt.title("US consumer price index (CPI) and Federal Reserve funds effective rate since 2020")
plt.xlabel("Date")
plt.ylabel("Rate (%)")
plt.legend()

plt.show()

In [ ]:
# When inflation quickly began to rise
cpi_rise_start = df_cpi_2020.loc[df_cpi_2020["rate"].diff() > 0.8].index[0].strftime("%Y-%m")
df_cpi_2020.loc[cpi_rise_start].round(2)

In [ ]:
# When inflation rate peaked
df_cpi_2020.loc[df_cpi_2020["rate"].idxmax().strftime("%Y-%m")]

In [ ]:
# When inflation started to become sticky
# Reverse df
df_cpi_2020_rev = df_cpi_2020.iloc[::-1].copy()
# Get its moving std
df_cpi_2020_rev["rate_moving_std"] = df_cpi_2020_rev["rate"].rolling(window=10).std()
# Get that reversed moving std (std of the next year-months) on the original df
df_cpi_2020["reverse_rate_moving_std"] = df_cpi_2020_rev["rate_moving_std"].iloc[::-1].values
# Get when reverse moving std goes above 0.3
cpi_sticky_start = df_cpi_2020[df_cpi_2020["reverse_rate_moving_std"] < 0.3].index[0].strftime("%Y-%m")
df_cpi_2020.loc[cpi_sticky_start][["rate"]]

In [ ]:
# Average inflation since started to become sticky
df_cpi[cpi_sticky_start:]["rate"].mean().round(2)

In [ ]:
# When the fed rate hikes started
fed_rise_start = df_fed_2020.loc[df_fed_2020["rate"].diff() > 0.1].index[0].strftime("%Y-%m")
df_fed_2020.loc[fed_rise_start]

In [ ]:
# When the fed big rate hikes started
fed_big_rise_start = df_fed_2020.loc[df_fed_2020["rate"].diff() > 0.4].index[0].strftime("%Y-%m")
df_fed_2020.loc[fed_big_rise_start]

In [ ]:
# When fed rate peaked
df_fed_2020.loc[df_fed_2020["rate"].idxmax().strftime("%Y-%m")]

In [ ]:
# Average difference between rates since inflation started to become sticky
(df_fed[cpi_sticky_start:]["rate"].mean() - df_cpi[cpi_sticky_start:]["rate"].mean()).round(2)

- As we can see CPI went above 2% on march of 2021, and the next month jumped to 4.16, peaking at about 9% on june of 2022
- the fed took about a year to start rising the rates, when inflation was no longer perceived as transitory.
- rates increased very slowly at first on march of 22, and on may started rising fast, until it peaked on august of 2023, staynin the same since.
- inflation came down until june of 2023, where it stabilized at about 3,3% (still above 2% mark) and became "sticky" since. 
- about 2 percentage points on average the diff